In [ ]:
!pip install transformers

!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.0.0+cpu.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.0.0+cpu.html
!pip install torch-geometric

!pip install scikit-learn
!pip install sparsemax
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-2.0+cu117.html


Looking in links: https://data.pyg.org/whl/torch-2.0.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.0/494.0 kB 4.3 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:

In [19]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaModel
import torch.nn as nn


In [9]:
from google.colab import files
uploaded = files.upload()

Saving amazon_reviews.csv to amazon_reviews.csv


In [22]:
df = pd.read_csv("amazon_reviews.csv")
df = df.sample(500)

df = df.dropna(subset=["text", "rating"])
df = df.reset_index(drop=True)

def combine_review_and_rating(row):
    review = str(row["text"])
    rating = str(row["rating"])
    return f"Review: {review}. Rating: {rating} stars."

df["combined_input"] = df.apply(combine_review_and_rating, axis=1)

df["label"] = (df["rating"] >= 4).astype(int)


df[["combined_input", "label"]].head()


,combined_input,label
0,Review: Headphones stay in place. Great for 3-...,1
1,Review: It’s beautiful but won’t stand up well...,0
2,"Review: Great product, works as advertised. Ra...",1
3,Review: From no signal to full signal as soon ...,1
4,Review: Very disappointed. Bought these for m...,0


In [16]:
import torch
from transformers import RobertaTokenizer, RobertaModel


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
roberta = RobertaModel.from_pretrained("roberta-base").to(device)
roberta.eval()


Using device: cpu


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dr

In [23]:
from torch.utils.data import Dataset, DataLoader

class CombinedReviewDataset(Dataset):
    def __init__(self, dataframe, tokenizer, roberta, max_len=128, stride=64, final_seq_len=128):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.roberta = roberta
        self.max_len = max_len
        self.stride = stride
        self.final_seq_len = final_seq_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = str(self.data.loc[idx, "combined_input"])
        label = self.data.loc[idx, "label"]


        encoded = self.tokenizer(
            text,
            return_overflowing_tokens=True,
            max_length=self.max_len,
            stride=self.stride,
            truncation=True,
            return_tensors="pt"
        )


        with torch.no_grad():
            chunks = []
            for i in range(encoded["input_ids"].shape[0]):
                input_ids = encoded["input_ids"][i].unsqueeze(0).to(self.roberta.device)
                attention_mask = encoded["attention_mask"][i].unsqueeze(0).to(self.roberta.device)
                output = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
                chunks.append(output.last_hidden_state.squeeze(0))

            full_embedding = torch.cat(chunks, dim=0)


        if full_embedding.shape[0] < self.final_seq_len:
            pad = torch.zeros(self.final_seq_len - full_embedding.shape[0], full_embedding.shape[1]).to(full_embedding.device)
            full_embedding = torch.cat([full_embedding, pad], dim=0)
        else:
            full_embedding = full_embedding[:self.final_seq_len]

        return {
            "embedding": full_embedding,
            "label": torch.tensor(label, dtype=torch.float)
        }


dataset = CombinedReviewDataset(df, tokenizer, roberta)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)


In [24]:
class BiGRU_MHSA_Classifier(nn.Module):
    def __init__(self, input_dim=768, hidden_dim=128, attention_heads=4, dropout=0.3):
        super(BiGRU_MHSA_Classifier, self).__init__()
        self.bigru = nn.GRU(input_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.attn = nn.MultiheadAttention(embed_dim=hidden_dim*2, num_heads=attention_heads, batch_first=True)
        self.fc1 = nn.Linear(hidden_dim * 2, 64)
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):

        gru_out, _ = self.bigru(x)
        attn_out, _ = self.attn(gru_out, gru_out, gru_out)
        pooled = attn_out.mean(dim=1)
        x = self.dropout(torch.relu(self.fc1(pooled)))
        return self.sigmoid(self.fc2(x)).squeeze()


In [25]:
import torch.optim as optim
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


model = BiGRU_MHSA_Classifier().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=2e-5)

train_losses = []


EPOCHS = 2
model.train()
for epoch in range(EPOCHS):
    epoch_loss = 0
    for batch in dataloader:
        inputs = batch["embedding"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(dataloader)
    train_losses.append(avg_loss)
    print(f"Epoch [{epoch+1}/{EPOCHS}] - Loss: {avg_loss:.4f}")


Epoch [1/2] - Loss: 0.6836
Epoch [2/2] - Loss: 0.6231
